In [5]:
import pandas as pd
import requests

matches = pd.read_csv('WCMatches.csv')
matches.columns
matches[["winnerid","loserid"]]




,winnerid,loserid
0,7d05pfwn,jfvflnna
1,o8q1tnkp,o8q1tnkp
2,akq5t76h,akq5t76h
3,ng9mpb4s,ng9mpb4s
4,ng9mpb4s,ng9mpb4s
...,...,...
2783,dq44xncy,222vn7x7
2784,g7umra42,g7umra42
2785,g7umra42,222vn7x7
2786,g7umra42,dq44xncy


In [16]:
unique_ids = pd.concat([matches['winnerid'], matches['loserid']]).unique()
print(unique_ids.size)

1210


In [ ]:
import time

player_info = pd.DataFrame()
for player_id in unique_ids[:10]:
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()
    
    # Combine data into a single dictionary for the player
    player_data = {
        "player_id": player_id,
        "events": events_data,
        "matches": matches_data
    }
    # Append as a new row
    player_info = pd.concat([player_info, pd.DataFrame([player_data])], ignore_index=True)
    time.sleep(0.5) #wait 0.5 second to not pummel the server


player_info.to_csv("EloPlayerinfo.csv")
print(player_info)

  player_id                                             events  \
0  7d05pfwn  {'data': [{'code': 'ptla98', 'cut_before': '8....   
1  o8q1tnkp  {'data': [{'code': 'ptla9899', 'cut_before': '...   
2  akq5t76h  {'data': [{'code': 'ptla98', 'cut_before': '8....   
3  ng9mpb4s  {'data': [{'code': 'ptla98', 'cut_before': '8....   
4  i8olnjut  {'data': [{'code': 'ptrome98', 'cut_before': '...   
5  7h18qwbk  {'data': [{'code': 'gpkc99', 'cut_before': '7....   
6  h0x63109  {'data': [{'code': 'ptla9899', 'cut_before': '...   
7  xva9s4ab  {'data': [{'code': 'gpsd99', 'cut_before': '8....   
8  dabm23qk  {'data': [{'code': 'gpdc99', 'cut_before': '8....   
9  79nnamgm  {'data': [{'code': 'ptla98', 'cut_before': '8....   

                                             matches  
0  {'ptla98': [{'match_id': 13, 'round': '1', 'ta...  
1  {'ptla9899': [{'match_id': 5717, 'round': '1',...  
2  {'ptla98': [{'match_id': 42, 'round': '1', 'ta...  
3  {'ptla98': [{'match_id': 87, 'round': '1', 'ta... 